# GAN for Generating Human Faces (CelebA)

In this notebook, we will build and train a simple Generative Adversarial Network (GAN) to generate human face images based on the CelebA dataset. Each step includes detailed explanations to help you understand how the model works.

## 1. Imports and Hyperparameters

We import necessary libraries and define hyperparameters:

- `tensorflow` and `keras` for model building and training.
- `tensorflow_datasets` to load CelebA.
- `matplotlib` and `numpy` for visualization and numerical operations.

In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import numpy as np

# Hyperparameters
latent_dim = 100        # Dimension of the random noise vector
img_height = 64         # Target image height
img_width = 64          # Target image width
channels = 3            # RGB channels
img_shape = (img_height, img_width, channels)
batch_size = 128        # Number of images per batch
epochs = 5000           # Total training steps (batches)
sample_interval = 500   # Interval to generate sample images

ModuleNotFoundError: No module named 'tensorflow_datasets'

## 2. Load and Preprocess the CelebA Dataset

We load CelebA via `tensorflow_datasets`, resize images to 64×64, and normalize pixel values to the range [-1, 1] to match the `tanh` output of the generator.

In [ ]:
def preprocess(sample):
    # Convert uint8 image to float32 in [-1,1]
    image = tf.image.resize(sample['image'], [img_height, img_width])
    image = (tf.cast(image, tf.float32) - 127.5) / 127.5
    return image

# Load dataset
ds = tfds.load('celeb_a', split='train', shuffle_files=True)
ds = ds.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)
ds = ds.shuffle(10000).batch(batch_size).prefetch(tf.data.AUTOTUNE)
print('Dataset ready:', ds)

## 3. Build the Generator Model

The generator will transform a random noise vector into a 64×64×3 image. We use:

- `Dense` and `Reshape` to project noise into a feature map.
- `BatchNormalization` and `LeakyReLU` for stable training.
- `Conv2DTranspose` layers to upsample to 64×64.
- `tanh` activation to output images in [-1, 1].

In [ ]:
def build_generator():
    model = models.Sequential(name='Generator')
    # project and reshape
    model.add(layers.Dense(8*8*256, input_dim=latent_dim))
    model.add(layers.Reshape((8, 8, 256)))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU(alpha=0.2))

    # upsample to 16x16
    model.add(layers.Conv2DTranspose(128, kernel_size=4, strides=2, padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU(alpha=0.2))

    # upsample to 32x32
    model.add(layers.Conv2DTranspose(64, kernel_size=4, strides=2, padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU(alpha=0.2))

    # upsample to 64x64
    model.add(layers.Conv2DTranspose(channels, kernel_size=4, strides=2, padding='same',
                                     activation='tanh'))
    return model

generator = build_generator()
generator.summary()

## 4. Build the Discriminator Model

The discriminator is a binary classifier that distinguishes real from generated faces. We use:

- `Conv2D` layers to extract features and downsample.
- `LeakyReLU` for non-linearity.
- `Dropout` to regularize.
- `Dense` with `sigmoid` for final probability output.

In [ ]:
def build_discriminator():
    model = models.Sequential(name='Discriminator')
    model.add(layers.Conv2D(64, kernel_size=4, strides=2, padding='same',
                             input_shape=img_shape))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Dropout(0.3))

    model.add(layers.Conv2D(128, kernel_size=4, strides=2, padding='same'))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Dropout(0.3))

    model.add(layers.Conv2D(256, kernel_size=4, strides=2, padding='same'))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Dropout(0.3))

    model.add(layers.Flatten())
    model.add(layers.Dense(1, activation='sigmoid'))
    return model

discriminator = build_discriminator()
discriminator.summary()

## 5. Compile the Discriminator and GAN

Compile discriminator with binary crossentropy and Adam optimizer. Then build the GAN by connecting generator and discriminator, freezing discriminator weights for generator training.

In [ ]:
discriminator.compile(optimizer=tf.keras.optimizers.Adam(0.0002, 0.5),
                       loss='binary_crossentropy',
                       metrics=['accuracy'])

z = layers.Input(shape=(latent_dim,))
img = generator(z)
discriminator.trainable = False
validity = discriminator(img)
gan = models.Model(z, validity)
gan.compile(optimizer=tf.keras.optimizers.Adam(0.0002, 0.5),
            loss='binary_crossentropy')
gan.summary()

## 6. Training the GAN

We train with alternating steps:
1. Train discriminator on real and fake images.
2. Train generator via GAN with labels as real to fool discriminator.
We periodically sample generated faces to visualize progress.

In [ ]:
import time

def sample_images(step, n=5):
    noise = np.random.normal(0, 1, (n, latent_dim))
    gen_imgs = generator.predict(noise)
    gen_imgs = 0.5 * gen_imgs + 0.5  # scale to [0,1]
    fig, axes = plt.subplots(1, n, figsize=(n*2,2))
    for i, ax in enumerate(axes):
        ax.imshow(gen_imgs[i])
        ax.axis('off')
    plt.show()

def train(epochs, batch_size=128):
    half = batch_size // 2
    for step in range(epochs):
        # Train Discriminator
        imgs = next(iter(ds))  # real images batch
        noise = np.random.normal(0, 1, (half, latent_dim))
        fake = generator.predict(noise)
        d_loss_real = discriminator.train_on_batch(imgs, np.ones((half,1)))
        d_loss_fake = discriminator.train_on_batch(fake, np.zeros((half,1)))
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

        # Train Generator
        noise = np.random.normal(0, 1, (batch_size, latent_dim))
        g_loss = gan.train_on_batch(noise, np.ones((batch_size,1)))

        # Display progress
        if step % sample_interval == 0:
            print(f"Step {step} [D loss: {d_loss[0]:.4f}] [G loss: {g_loss:.4f}]")
            sample_images(step)

## 7. Run Training

Start training. Be patient, as generating high-quality faces takes time.

In [ ]:
train(epochs=epochs, batch_size=batch_size)

## Summary and Next Steps

- We built a GAN on CelebA to generate 64×64 RGB faces.
- Each step was explained, from data loading to training loop.
- Next, try increasing model depth, adding residual blocks, or using data augmentation.